In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
datapool = pd.read_excel(r"C:\Users\SnappFood\Desktop\embedding address\datapool.xlsx")
leadcollection = pd.read_excel(r"C:\Users\SnappFood\Desktop\embedding address\lead collection.xlsx")

In [5]:
datapool.dropna(inplace=True)
leadcollection.dropna(inplace=True)

In [6]:
def clean_name(name):
    name = str(name).replace("آ", "ا").replace("ي", "ی").replace("ك", "ک").replace("\u200c", "").replace("ئ", "ی").replace(".", "").replace(" ", "").replace("‌", "")
    name = re.sub(r'\(.*?\)', '', name)
    name = name.strip()
    return name

In [7]:
datapool["Clean Name"] = datapool["Name"].apply(clean_name)
leadcollection["Clean Name"] = leadcollection["Name"].apply(clean_name)

In [8]:
datapool['Address'] = datapool['Address'].astype(str)
leadcollection['Address'] = leadcollection['Address'].astype(str)

In [9]:
datapool['combined'] = datapool['Clean Name'] + ' ' + datapool['Address']
leadcollection['combined'] = leadcollection['Clean Name'] + ' ' + leadcollection['Address']

In [10]:
vectorizer = TfidfVectorizer()
all_texts = datapool['combined'].to_list() + leadcollection['combined'].to_list()
vectorizer.fit(all_texts)

TfidfVectorizer()

In [11]:
tfidf_matrix_1 = vectorizer.transform(datapool["combined"])
tfidf_matrix_2 = vectorizer.transform(leadcollection["combined"])

In [12]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix_1, tfidf_matrix_2)

# Convert to a DataFrame for better visualization, if needed
similarity_df = pd.DataFrame(similarity_matrix, index=datapool.index, columns=leadcollection.index)


In [14]:
threshold = 0.7

# Convert columns to numpy arrays for faster access
datapool_city = datapool['City'].values
leadcollection_city = leadcollection['City'].values
datapool_ids = datapool['ID'].values
datapool_names = datapool['Name'].values

# Initialize the list to store the most similar pairs
most_similar_pairs = []

for i in range(similarity_matrix.shape[1]):
    # Get the indices where the cities match
    matching_indices = np.where(leadcollection_city[i] == datapool_city)[0]
    
    if len(matching_indices) == 0:
        continue  # Skip if no matching cities are found

    # Extract the relevant similarity scores
    similarities = similarity_matrix[matching_indices, i]
    # Find the index of the maximum similarity score
    max_idx = np.argmax(similarities)
    max_similarity = similarities[max_idx]
    
    if max_similarity > threshold:
        j = matching_indices[max_idx]
        leadcollection.at[i, 'ID'] = datapool_ids[j]
        leadcollection.at[i, 'ratio'] = max_similarity
        most_similar_pairs.append((datapool_names[j], leadcollection.iloc[i]['Name']))


In [15]:
leadcollection.drop(columns = ["Clean Name", "combined"], inplace=True)

In [16]:
leadcollection.to_excel(r"C:\Users\SnappFood\Desktop\embedding address\lead collection final.xlsx", index=False)

In [20]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_data(datapool_path, leadcollection_path):
    datapool = pd.read_excel(datapool_path)
    leadcollection = pd.read_excel(leadcollection_path)
    datapool.dropna(inplace=True)
    leadcollection.dropna(inplace=True)
    return datapool, leadcollection

def clean_name(name):
    name = str(name).replace("آ", "ا").replace("ي", "ی").replace("ك", "ک").replace("\u200c", "").replace("ئ", "ی").replace(".", "").replace(" ", "").replace("\u200c", "")
    name = re.sub(r'\(.*?\)', '', name)
    name = name.strip()
    return name

def preprocess_data(datapool, leadcollection):
    datapool["Clean Name"] = datapool["Name"].apply(clean_name)
    leadcollection["Clean Name"] = leadcollection["Name"].apply(clean_name)

    datapool['Address'] = datapool['Address'].astype(str)
    leadcollection['Address'] = leadcollection['Address'].astype(str)

    datapool['combined'] = datapool['Clean Name'] + ' ' + datapool['Address']
    leadcollection['combined'] = leadcollection['Clean Name'] + ' ' + leadcollection['Address']
    
    return datapool, leadcollection

def fit_vectorizer(datapool, leadcollection):
    vectorizer = TfidfVectorizer()
    all_texts = datapool['combined'].to_list() + leadcollection['combined'].to_list()
    vectorizer.fit(all_texts)
    return vectorizer

def compute_similarity(vectorizer, datapool, leadcollection):
    tfidf_matrix_1 = vectorizer.transform(datapool["combined"])
    tfidf_matrix_2 = vectorizer.transform(leadcollection["combined"])
    similarity_matrix = cosine_similarity(tfidf_matrix_1, tfidf_matrix_2)
    similarity_df = pd.DataFrame(similarity_matrix, index=datapool.index, columns=leadcollection.index)
    return similarity_df

def match_addresses(datapool, leadcollection, similarity_matrix, threshold=0.7):
    datapool_city = datapool['City'].values
    leadcollection_city = leadcollection['City'].values
    datapool_ids = datapool['ID'].values
    datapool_names = datapool['Name'].values

    most_similar_pairs = []

    for i in range(similarity_matrix.shape[1]):
        # Ensure that i is an integer index and that cities match correctly
        matching_indices = np.where(leadcollection_city[i] == datapool_city)[0]
        
        if len(matching_indices) == 0:
            continue

        # Extract the relevant similarity scores and prevent index errors
        similarities = similarity_matrix[matching_indices, i]
        if similarities.size == 0:
            continue

        max_idx = np.argmax(similarities)
        max_similarity = similarities[max_idx]
        
        if max_similarity > threshold:
            j = matching_indices[max_idx]
            leadcollection.at[i, 'ID'] = datapool_ids[j]
            leadcollection.at[i, 'ratio'] = max_similarity
            
    return leadcollection

def clean_up(leadcollection):
    leadcollection.drop(columns = ["Clean Name", "combined"], inplace=True)
    return leadcollection

def save_results(leadcollection, output_path):
    leadcollection.to_excel(output_path, index=False)

# Example usage
datapool_path = r"C:\\Users\\SnappFood\\Desktop\\embedding address\\datapool.xlsx"
leadcollection_path = r"C:\\Users\\SnappFood\\Desktop\\embedding address\\lead collection.xlsx"
output_path = r"C:\\Users\\SnappFood\\Desktop\\embedding address\\lead collection final1.xlsx"

datapool, leadcollection = load_data(datapool_path, leadcollection_path)
datapool, leadcollection = preprocess_data(datapool, leadcollection)
vectorizer = fit_vectorizer(datapool, leadcollection)
similarity_matrix = compute_similarity(vectorizer, datapool, leadcollection)
leadcollection = match_addresses(datapool, leadcollection, similarity_matrix)
leadcollection = clean_up(leadcollection)
save_results(leadcollection, output_path)


KeyError: (array([    3,     4,     7, ..., 46275, 46278, 46285], dtype=int64), 0)